### image_shower.ipynb

In [ ]:
import os
import pdb
import json
import PIL
from glob import glob
# from pipelines.video_action_recognition_config import get_cfg_defaults
# from attention_playground import make_transforms, load_annotation, loadvideo
import csv
import visualization_utils_custom as vis_utils
import imageio.v2 as imageio
from IPython import display
import random
import numpy as np
import matplotlib.pyplot as plt


dataset = "ucf"
gpu_num = 4
if dataset == "jhmdb":
    gt = '/mnt/video_nfs4/users/jinsung/results/tubelet-transformer/JHMDB_Tuber/Dab_conv_trans_decsasde_CSN152_270-420/res/GT_{}.txt'
    detection = '/mnt/video_nfs4/users/jinsung/results/tubelet-transformer/JHMDB_Tuber/Dab_conv_trans_decsasde_CSN152_270-420/res/{}.txt'
else:
    gt = '/mnt/video_nfs4/users/jinsung/results/tubelet-transformer/UCF/Dab_conv_trans_decsasde_CSN152_270-416/res/GT_{}.txt'
    detection = '/mnt/video_nfs4/users/jinsung/results/tubelet-transformer/UCF/Dab_conv_trans_decsasde_CSN152_270-416/res/{}.txt'

UCF_CLASSES = ['Basketball', 'BasketballDunk', 'Biking', 'CliffDiving', 'CricketBowling', 'Diving', 'Fencing',
        'FloorGymnastics', 'GolfSwing', 'HorseRiding', 'IceDancing', 'LongJump', 'PoleVault', 'RopeClimbing',
        'SalsaSpin','SkateBoarding', 'Skiing', 'Skijet', 'SoccerJuggling',
        'Surfing', 'TennisSwing', 'TrampolineJumping', 'VolleyballSpiking', 'WalkingWithDog']

JHMDB_CLASSES = ['brush_hair', 'catch', 'clap', 'climb_stairs', 'golf', 'jump', 'kick_ball', 'pick', 'pour', 'pullup',
           'push', 'run', 'shoot_ball', 'shoot_bow', 'shoot_gun', 'sit', 'stand', 'swing_baseball', 'throw',
           'walk', 'wave']

items = {}
if dataset == "ucf":
    for i, c in enumerate(UCF_CLASSES):
        items[i] = c
else:
    for i, c in enumerate(JHMDB_CLASSES):
        items[i] = c        

frame_list = {}
path_list = {}

plt.figure()
%matplotlib inline

for i in range(gpu_num):
    with open(gt.format(i)) as f:
        for line in f.readlines():
            two_let = False
            img_id = line.split(' ')[0] 
            # JHMDB: sit_HP_PRISONER_OF_AZKABAN_sit_f_cm_np1_le_med_1_00
            # (path: /mnt/tmp/JHMDB/Frames/sit/HP_PRISONER_OF_AZKABAN_sit_f_cm_np1_le_med_1/00001.png)
            # UCF: Surfing_v_Surfing_g01_c04_063 
            # (path: /mnt/tmp/UCF101_v2/rgb-images/Surfing/v_Surfing_g01_c04/00063.jpg)
            cat = img_id.split("_")[0]
            if dataset == "jhmdb":
                if cat in ["brush", "climb", "kick", "shoot", "swing"]:
                    two_let = True
                    cat = "_".join(img_id.split("_")[:2])
            if two_let:
                vid = "_".join(img_id.split("_")[2:-1])
            else:
                vid = "_".join(img_id.split("_")[1:-1])
            fid = int(img_id.split("_")[-1])
            if not cat in frame_list.keys():
                frame_list[cat] = []
                path_list[cat] = []
            frame_list[cat].append(img_id)
            if dataset == "jhmdb":
                fid = f"{fid+1:05d}"
                path_list[cat].append(f"/mnt/tmp/JHMDB/Frames/{cat}/{vid}/{fid}.png")
            else:
                fid = f"{fid:05d}"
                path_list[cat].append(f"/mnt/tmp/UCF101_v2/rgb-images/{cat}/{vid}/{fid}.jpg")

label = random.randint(1,len(items)-1)
# label = 10
selected_cat = items[label]

go_on = True
while go_on:
    ind = random.randint(0, len(path_list[selected_cat])-1)
    path = path_list[selected_cat][ind]
    img = PIL.Image.open(path)
    w, h = img.size
    if w >= h:
        nh = 224
        nw = 224 * w / h
    else:
        nw = 224
        nh = 224 * h / w

    img = img.resize((int(nw), int(nh)))
    plt.imshow(img)
    plt.show()
    response = input()
    
    if response == "y":
        go_on = False
    fid = frame_list[selected_cat][ind]

selected_vid = "_".join(fid.split("_")[1:-1])
vid_dir = "/".join(path.split("/")[:-1])
video_list = os.listdir(vid_dir)

# vid_name = "v_Basketball_g07_c01"
# vid_name = "v_Skijet_g03_c01"
# vid_name = "v_TennisSwing_g07_c05"
# vid_name = "HP_PRISONER_OF_AZKABAN_sit_f_cm_np1_le_med_1"
# vid_name = "Torwarttraining_catch_u_cm_np1_ri_med_0"
# vid_name = "pushing_cart_push_f_cm_np1_le_bad_0"
# dataset_path = '/mnt/tmp/UCF101_v2/rgb-images/Basketball/{}'.format(vid_name)
# dataset_path = '/mnt/tmp/UCF101_v2/rgb-images/Skijet/{}'.format(vid_name)
# dataset_path = '/mnt/tmp/UCF101_v2/rgb-images/TennisSwing/{}'.format(vid_name)
# # dataset_path = '/mnt/tmp/JHMDB/Frames/sit/{}'.format(vid_name)
# video_list = os.listdir(dataset_path)
vis_save_path = './output_images'

save_path = "./"

# detection = '../tmp_jhmdb/0.txt' #numbers are changeable    

# gt = '../ucf_gt/GT_1.txt'
# gt = '../jhmdb_gt/GT_0.txt'


frame_keys_dict = {}

anno_dict = {}
detection_dict = {}
for i in range(gpu_num):
    with open(gt.format(i)) as f:
        for line in f.readlines():
            img_id = line.split(' ')[0]
            if not selected_vid in img_id: continue
            annotation = [int(float(n)) for n in line.split('[')[1].split(']')[0].split(',')]
            one_hot_obj_label = annotation[6:]
            try:
                obj_labels = [items[i] for i, e in enumerate(one_hot_obj_label) if e!=0]
            except:
                obj_labels = [items[i] for i, e in enumerate(one_hot_obj_label[:-1]) if e!=0]
            coord = annotation[2:6]
            if img_id not in anno_dict.keys():
                anno_dict[img_id] = {
                    "obj": [obj_labels],
                    "coord": [coord]
                }
            else:
                anno_dict[img_id]["obj"].append(obj_labels)
                anno_dict[img_id]["coord"].append(coord)
            
for i in range(gpu_num):
    with open(detection.format(i)) as f:
        for line in f.readlines():
            img_id = line.split(' ')[0]
            if not selected_vid in img_id: continue
            annotation = [float(n) for n in line.split('[')[1].split(']')[0].split(',')]
            one_hot_obj_label = annotation[4:-1]
            is_not_valid = annotation[-1]
            if is_not_valid > 0.8:
                continue
            # obj_labels = [items[i] for i, e in enumerate(one_hot_obj_label) if e!=0]
            obj_labels = items[np.array(one_hot_obj_label).argmax()]
            coord = annotation[:4]
            if img_id not in detection_dict.keys():
                detection_dict[img_id] = {
                    "obj": [obj_labels],
                    "coord": [coord]
                }
            else:
                detection_dict[img_id]["obj"].append(obj_labels)
                detection_dict[img_id]["coord"].append(coord)            


def crop_normalize(img, bboxes):
    imgs = list()
    for box in bboxes:
        h, w = img.size
        left, top, right, bottom = w * box[0], h * box[1], w * box[2], h * box[3]
        img.crop((left, top, right, bottom)).resize(128, 128)
        imgs.append()

    return imgs


def find_key_frame(vid, frame_id):
    for i, key_frame in enumerate(frame_keys_dict[vid]): # key_frame is sorted in ascending order
        if key_frame*30-27000 < frame_id:
            continue
        else:
            keyf_index = i
            break
    try:
        if (frame_keys_dict[vid][keyf_index]*30-27000 - frame_id) < (frame_id - (frame_keys_dict[vid][max(keyf_index-1, 0)]*30-27000)):
            return frame_keys_dict[vid][keyf_index]
        else: 
            return frame_keys_dict[vid][max(keyf_index-1, 0)]
    except:
        return frame_keys_dict[vid][0]


# random_vid = random.randint(0,len(video_list)-1)

def img_along_(t, cat, vid_dir, vid, dataset):
    # for video in video_list:
    video_frame_path = vid_dir
    if dataset == "jhmdb":
        video_frame_list = sorted(glob(video_frame_path + '/*.png'))
    else:
        video_frame_list = sorted(glob(video_frame_path + '/*.jpg'))
    # for frame in video_frame_list:
    frame = video_frame_list[t]
    img = PIL.Image.open(video_frame_path + "/" + frame.split('/')[-1])
    w, h = img.size
    if w >= h:
        nh = 224
        nw = 224 * w / h
    else:
        nw = 224
        nh = 224 * h / w

    img = img.resize((int(nw), int(nh)))
    # if t >=95 and t <= 148:
    if True:
        # frame_id = int(frame[-10:-4])
        # reference_key_frame = find_key_frame(video, frame_id)
        # vid_keyf = video + "," + str(reference_key_frame).zfill(4)
        # bboxes = frame_bbox[vid_keyf]["bboxes"]
        # acts = frame_bbox[vid_keyf]["acts"]
        if dataset == "jhmdb":
            n = t
            bboxes = anno_dict["_".join([cat, vid]) + f"_{n:02d}"]["coord"]
        else:
            n = t+1
            try:
                bboxes = anno_dict["_".join([cat, vid]) + f"_{n:03d}"]["coord"]
            except:
                bboxes = []
        # acts = anno_dict["_".join([cat, vid]) + f"_{n:02d}"]["obj"]
        for i, coord in enumerate(bboxes):
            xmin, ymin, xmax, ymax = coord[0], coord[1], coord[2], coord[3]
            cat_list = [cat]
            vis_utils.draw_bounding_box_on_image(
                img, ymin, xmin, ymax, xmax,
                color = 'Green',
                display_str_list=cat_list,
                use_normalized_coordinates=False
            )
        if dataset == "jhmdb":
            det_bboxes = detection_dict["_".join([cat, vid]) + f"_{n:02d}"]["coord"]
            det_acts = detection_dict["_".join([cat, vid]) + f"_{n:02d}"]["obj"]
        else:
            try:
                det_bboxes = detection_dict["_".join([cat, vid]) + f"_{n:03d}"]["coord"]
                det_acts = detection_dict["_".join([cat, vid]) + f"_{n:03d}"]["obj"]
            except:
                det_bboxes = []
        for i, coord in enumerate(det_bboxes):
            xmin, ymin, xmax, ymax = coord[0], coord[1], coord[2], coord[3]
            vis_utils.draw_bounding_box_on_image(
                img, ymin, xmin, ymax, xmax,
                color = 'yellow',
                display_str_list=det_acts,
                use_normalized_coordinates=False
            )            
            # display(img)
        
        return img
    else:
        return img

In [ ]:
ims = []
for t in range(len(video_list)-1): # this decides video length
    ims.append(img_along_(t, selected_cat, vid_dir, selected_vid, dataset))

imageio.mimsave('./temp.gif', ims, fps=25)

display.Image("./temp.gif")

In [ ]:
import numpy as np
import itertools

train_co_occurence_map = np.zeros((80,80))
val_co_occurence_map = np.zeros((80,80))

for key in train_frame_keys_list:
    instance_list = train_video_frame_bbox[key]['acts']
    for instance in instance_list:
        # instance: [m, n, o]
        for i, j in itertools.product(instance, instance):
            train_co_occurence_map[i, j] += 1

for key in val_frame_keys_list:
    instance_list = val_video_frame_bbox[key]['acts']
    for instance in instance_list:
        # instance: [m, n, o]
        for i, j in itertools.product(instance, instance):
            val_co_occurence_map[i, j] += 1

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

train_co_occurence_map = sigmoid(train_co_occurence_map)
val_co_occurence_map = sigmoid(val_co_occurence_map)


output_labels = []
for gpu_num in range(4):
    detection = '/mnt/video-nfs5/users/jinsung/results/tubelet-transformer/eval/tmp3/{}.txt'.format(gpu_num)
    with open(detection) as f:
        for line in f.readlines():
            # img_id = line.split(' ')[0]
            annotation = [float(n) for n in line.split('[')[1].split(']')[0].split(',')]
            multi_hot_obj_label = annotation[4:-1]
            output_labels.append([i for i, e in enumerate([k>0.6 for k in multi_hot_obj_label]) if e])


model_output_co_occurence_map = np.zeros((80,80))

for instance in output_labels:
    # instance: [m, n, o]
    if len(instance)!=0:
        for i, j in itertools.product(instance, instance):
            model_output_co_occurence_map[i, j] += 1
    else:
        pass


model_output_co_occurence_map = sigmoid(model_output_co_occurence_map)

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline
fig, axs = plt.subplots(ncols=3, nrows=1)
axs[0].imshow(train_co_occurence_map, cmap='copper', interpolation=None, alpha=.7)
axs[0].set_title("train set")
axs[1].imshow(val_co_occurence_map, cmap='copper', interpolation=None, alpha=.7)
axs[1].set_title("val set")
axs[2].imshow(model_output_co_occurence_map, cmap='copper', interpolation=None, alpha=.7)
axs[2].set_title("model_output")